**SIZING OF 2 PHASE VERTICAL OIL AND GAS SEPARATOR**



In [42]:
import numpy as np

def gas_density(temp, pressure, sg, z):
  """
  Calculate Gas Density
  For range: this is not a correlation, so valid for infinite intervals
  """  
  temp = temp + 459.67
  R = 10.732 # gas constant in (ft3*psi)/(lb-mol*R) 
  rhogas = (28.97 * sg * pressure) / (z * R * temp)
  return rhogas 

def gas_capacity(temp,pressure,z_factor,GFR,rhogas,rhooil,Cd0,dm):
   d=((5040)*((temp+459.67)*z_factor*GFR/pressure)*((rhogas/(rhooil-rhogas)*(Cd0/dm)))**0.5)**0.5
   return d
 

def oil_density(OilAPI):
  rhooil = 62.4*(141.5/(131.5+OilAPI))
  return rhooil

def gas_pseudoprops(temp, pressure, sg, x_h2s, x_co2):
  """
  Calculate Gas Pseudo-critical and Pseudo-reduced Pressure and Temperature
  * Pseudo-critical properties
    For range: 0.57 < sg < 1.68
    (Sutton, 1985)
  * Pseudo-reduced properties
    For range: x_h2s (mol%) < 0.738; x_co2 (mol%) < 0.544; 154 < p (psia) < 7026; 40 < temp (°F) < 300 (error 0.97%)
    (Wichert and Aziz, 1972)
  """
  import numpy as np

  if sg > 0.57 and sg < 1.68 and x_h2s < 0.738 and x_co2 < 0.544 and pressure > 154 and pressure < 7026 and temp > 40 and temp < 300:
    temp = temp + 459.67 # convert to Rankine

    # calculate pseudocritical properties (Sutton, valid for 0.57<sg<1.68)
    P_pc = 756.8 - (131.07 * sg) - (3.6 * sg**2)
    T_pc = 169.2 + (349.50 * sg) - (74 * sg**2) # in Rankine

    # calculate adjustment to pseudocritical properties for sour gas (Wiechert-Aziz, valid for x_co2<0.544 and x_h2s<0.738)
    e = (120 * (((x_h2s + x_co2)**0.9) - ((x_h2s + x_co2)**1.6))) + (15 * (x_h2s**0.5 - x_h2s**4))
    T_pc = T_pc - e # corrected T_pc
    P_pc = (P_pc * T_pc) / (T_pc - x_h2s * e * (1-x_h2s))

    # calculate pseudoreduced properties
    P_pr = pressure / P_pc
    T_pr = temp / T_pc
  
  else:
    P_pc, T_pc, P_pr, T_pr = np.nan, np.nan, np.nan, np.nan

  return(P_pc, T_pc, P_pr, T_pr)

def gas_zfactor(T_pr, P_pr):
  """
  Calculate Gas Compressibility Factor
  For range: 0.2 < P_pr < 30; 1 < T_pr < 3 (error 0.486%)
  (Dranchuk and Aboukassem, 1975)
  """
  # T_pr : calculated pseudoreduced temperature
  # P_pr : calculated pseudoreduced pressure   
  from scipy.optimize import fsolve # non-linear solver
  import numpy as np

  if T_pr > 1 and T_pr < 3 and P_pr > 0.2 and P_pr < 30:
    a1 = 0.3265; a2 = -1.0700; a3 = -0.5339; a4 = 0.01569; a5 = -0.05165; a6 = 0.5475
    a7 = -0.7361; a8 = 0.1844; a9 = 0.1056; a10 = 0.6134; a11 = 0.7210

    def f(y):
      rho_pr, z = y
      c1 = a1 + (a2/T_pr) + (a3/(T_pr**3))+ (a4/(T_pr**4))+ (a5/(T_pr**5))
      c2 = a6 + (a7/T_pr) + (a8/(T_pr**2))
      c3 = a9*((a7/T_pr) + (a8/(T_pr**2)))
      c4 = (a10)*(1+(a11*(rho_pr**2)))*((rho_pr**2)/(T_pr**3))*(np.exp(-a11*(rho_pr**2)))

      f1 = z + (c3*(rho_pr**5)) - (c2*(rho_pr**2)) - (c1*(rho_pr**1)) - c4 - 1
      f2 = rho_pr - ((0.27 * P_pr) / (z * T_pr))
      return[f1, f2]

    pseudo_rho, z_factor = fsolve(f, [1, 1]) # initial guess
  
  else:
    pseudo_rho, z_factor = np.nan, np.nan

  return(pseudo_rho, z_factor) # result is density, z-factor


      
print('INPUT')
pressure = float(input('input the working pressure in PSI-   ')) # psi
temp = float(input('input the working Temperatue in Farenhite-   ')) # Fahrenheit
sg = float(input('input the Specific gravity of gas-    ')) # specific gravity
x_h2s =float(input('mole fraction of H2S in gas-    '))
x_co2 = float(input('mole fraction of CO2 in gas-    '))  
GFR = float(input('Gas Flow Rate in MMscfd-    ')) 
LFR = float(input('Liquid Flow Rate in  Bopd-    ')) 
viscogas= float(input('viscocity of gas in Cp-   '))
dm= float(input('drop diameter in microns-   '))
OilAPI= float(input('input the oil API Gravity-   '))




# calculate gas pseudoproperties 
P_pc, T_pc, P_pr, T_pr = gas_pseudoprops(temp, pressure, sg, x_h2s, x_co2)


# calculate gas z-factor 
pseudo_rho, z_factor = gas_zfactor(T_pr, P_pr)


# calculate gas density
rhogas = gas_density(temp, pressure, sg, z_factor)

# calculate OIL density
rhooil = oil_density(OilAPI)





#dischare coefficient calculations 
Cd0=0.34
t=None
while t != 0:
    Vt = 0.0119* ((rhooil-rhogas ) / rhogas * (dm/Cd0))**0.5
    Re1 = ((0.0049) * (dm*Vt*rhogas))/viscogas
    Cd1=(24/Re1)+(3/Re1**0.5)+0.34
    t=round(Cd0,4)- round(Cd1,4)
    Cd0=Cd1
   

    
    
#Calculate gas capacity constrants
d = gas_capacity(temp,pressure,z_factor,GFR,rhogas,rhooil,Cd0,dm)
#Re = (0.0049) * (dm*Vt*rhogas)/viscogas

 
print('OUTPUT')
print("The Compessibility factor of gas is equal to -  " ,round(z_factor,3))
print("The Viscocity of gas is equal to -  " ,round(viscogas,3))
print("The density of gas is equal to - " ,round(rhogas,3))
print("The density of oil is equal to - " ,round(rhooil,3))
print("The density of gas is equal to - " ,round(rhooil,3))
print("The gas capacity factor (d) is equal to - " ,round(d,3))
print("The discharge coefficient is equal to ",round(Cd0,3))

#Liqid capacity Contraints
tr=[0.5,1,2,3]
length0 = len(tr)
D=round(d)
dlist=[float(D),float(D)+5,float(D)+10,float(D)+15,float(D)+20]
length1 =len(dlist)

hlist=[]
Lsslist=[]
Slenr=[]
h=[]

for x in tr:
  print("For the residence time (Tr) =",x)
  for y in dlist:
    calc=float((x*LFR)/(0.12*(y**2)))
    h.append(calc)
    calc1=float((calc+76)/12)
    Lsslist.append(calc1)
    calc2=float(12*calc1/y)
    Slenr.append(calc2)
    print("The value of d(in)-   ",round(y,2),"   The value of h(in)-   ",round(h[-1],2),'   The value of Lss(ft)-    ',round(Lsslist[-1],2),"    The value of Slendernness Ratio-   ",round(Slenr[-1],2))

print('NOTE- The Slenderness ration of 3-4 is more preffered while choosing the appropriate design parameters')

INPUT
input the working pressure in PSI-   1000
input the working Temperatue in Farenhite-   60
input the Specific gravity of gas-    0.6
mole fraction of H2S in gas-    0
mole fraction of CO2 in gas-    0
Gas Flow Rate in MMscfd-    10
Liquid Flow Rate in  Bopd-    2000
viscocity of gas in Cp-   0.013
drop diameter in microns-   140
input the oil API Gravity-   40
OUTPUT
The Compessibility factor of gas is equal to -   0.852
The Viscocity of gas is equal to -   0.013
The density of gas is equal to -  3.657
The density of oil is equal to -  51.485
The density of gas is equal to -  51.485
The gas capacity factor (d) is equal to -  21.977
The discharge coefficient is equal to  0.857
For the residence time (Tr) = 0.5
The value of d-    22.0    The value of h-    17.22    The value of Lss-     7.77     The value of Slendernness Ratio-    4.24
The value of d-    27.0    The value of h-    11.43    The value of Lss-     7.29     The value of Slendernness Ratio-    3.24
The value of d-    32.

# **SIZING OF A HORIZONTAL 2 PHASE SEPERATOR**

In [30]:
import numpy as np
import pandas as pd 

def gas_density(temp, pressure, sg, z):
  """
  Calculate Gas Density
  For range: this is not a correlation, so valid for infinite intervals
  """  
  temp = temp + 459.67
  R = 10.732 # gas constant in (ft3*psi)/(lb-mol*R) 
  rhogas = (28.97 * sg * pressure) / (z * R * temp)
  return rhogas 

def gas_capacity(temp,pressure,z_factor,GFR,rhogas,rhooil,Cd0,dm):
   dLeff=((420)*((temp+460)*z_factor*GFR/pressure)*(((rhogas/(rhooil-rhogas))*(Cd0/dm))**0.5))
   return dLeff
 


def oil_density(OilAPI):
  rhooil = 62.4*(141.5/(131.5+OilAPI))
  return rhooil

def gas_pseudoprops(temp, pressure, sg, x_h2s, x_co2):
  """
  Calculate Gas Pseudo-critical and Pseudo-reduced Pressure and Temperature
  * Pseudo-critical properties
    For range: 0.57 < sg < 1.68
    (Sutton, 1985)
  * Pseudo-reduced properties
    For range: x_h2s (mol%) < 0.738; x_co2 (mol%) < 0.544; 154 < p (psia) < 7026; 40 < temp (°F) < 300 (error 0.97%)
    (Wichert and Aziz, 1972)
  """
  import numpy as np

  if sg > 0.57 and sg < 1.68 and x_h2s < 0.738 and x_co2 < 0.544 and pressure > 154 and pressure < 7026 and temp > 40 and temp < 300:
    temp = temp + 459.67 # convert to Rankine

    # calculate pseudocritical properties (Sutton, valid for 0.57<sg<1.68)
    P_pc = 756.8 - (131.07 * sg) - (3.6 * sg**2)
    T_pc = 169.2 + (349.50 * sg) - (74 * sg**2) # in Rankine

    # calculate adjustment to pseudocritical properties for sour gas (Wiechert-Aziz, valid for x_co2<0.544 and x_h2s<0.738)
    e = (120 * (((x_h2s + x_co2)**0.9) - ((x_h2s + x_co2)**1.6))) + (15 * (x_h2s**0.5 - x_h2s**4))
    T_pc = T_pc - e # corrected T_pc
    P_pc = (P_pc * T_pc) / (T_pc - x_h2s * e * (1-x_h2s))

    # calculate pseudoreduced properties
    P_pr = pressure / P_pc
    T_pr = temp / T_pc
  
  else:
    P_pc, T_pc, P_pr, T_pr = np.nan, np.nan, np.nan, np.nan

  return(P_pc, T_pc, P_pr, T_pr)

def gas_zfactor(T_pr, P_pr):
  """
  Calculate Gas Compressibility Factor
  For range: 0.2 < P_pr < 30; 1 < T_pr < 3 (error 0.486%)
  (Dranchuk and Aboukassem, 1975)
  """
  # T_pr : calculated pseudoreduced temperature
  # P_pr : calculated pseudoreduced pressure   
  from scipy.optimize import fsolve # non-linear solver
  import numpy as np

  if T_pr > 1 and T_pr < 3 and P_pr > 0.2 and P_pr < 30:
    a1 = 0.3265; a2 = -1.0700; a3 = -0.5339; a4 = 0.01569; a5 = -0.05165; a6 = 0.5475
    a7 = -0.7361; a8 = 0.1844; a9 = 0.1056; a10 = 0.6134; a11 = 0.7210

    def f(y):
      rho_pr, z = y
      c1 = a1 + (a2/T_pr) + (a3/(T_pr**3))+ (a4/(T_pr**4))+ (a5/(T_pr**5))
      c2 = a6 + (a7/T_pr) + (a8/(T_pr**2))
      c3 = a9*((a7/T_pr) + (a8/(T_pr**2)))
      c4 = (a10)*(1+(a11*(rho_pr**2)))*((rho_pr**2)/(T_pr**3))*(np.exp(-a11*(rho_pr**2)))

      f1 = z + (c3*(rho_pr**5)) - (c2*(rho_pr**2)) - (c1*(rho_pr**1)) - c4 - 1
      f2 = rho_pr - ((0.27 * P_pr) / (z * T_pr))
      return[f1, f2]

    pseudo_rho, z_factor = fsolve(f, [1, 1]) # initial guess
  
  else:
    pseudo_rho, z_factor = np.nan, np.nan

  return(pseudo_rho, z_factor) # result is density, z-factor

print('INPUT')
      

pressure = float(input('input the working pressure in PSI-   ')) # psi
temp = float(input('input the working Temperatue in Farenhite-   ')) # Fahrenheit
sg = float(input('input the Specific gravity of gas-   '   )) # specific gravity
x_h2s =float(input('input the mole fraction of H2S in gas-   '))
x_co2 = float(input('input the mole fraction of CO2 in gas-   '  ))  
GFR = float(input('input the Gas Flow Rate in MMscfd -   '   )) 
LFR = float(input('input the iquid Flow Rate in  Bopd-     '   )) 
viscogas= float(input('input the viscocity of gas in Cp-   '   ))
dm= float(input('input the drop diameter in microns-  '   ))
OilAPI= float(input('input the oil API Gravity-  '   ))




# calculate gas pseudoproperties 
P_pc, T_pc, P_pr, T_pr = gas_pseudoprops(temp, pressure, sg, x_h2s, x_co2)


# calculate gas z-factor 
pseudo_rho, z_factor = gas_zfactor(T_pr, P_pr)


# calculate gas density
rhogas = gas_density(temp, pressure, sg, z_factor)

# calculate OIL density
rhooil = oil_density(OilAPI)



# Terminal velocity
#Vt = 0.0119* (rhooil-rhogas ) / rhogas * (dm/Cd)


#Calculate Reynolds Number
#Re = Reynolds_number(dm,Vt,rhogas,viscogas)
#Calculate Dsicharge coefficient
#Cd = (24/Re)+(3/Re**0.5)+0.34



Cd0=0.34
t=None
while t != 0:
    Vt = 0.0119* ((rhooil-rhogas ) / rhogas * (dm/Cd0))**0.5
    Re1 = ((0.0049) * (dm*Vt*rhogas))/viscogas
    Cd1=(24/Re1)+(3/Re1**0.5)+0.34
    t=round(Cd0,4)- round(Cd1,4)
    Cd0=Cd1
    
   
#Calculate gas capacity constrants
dLeff = gas_capacity(temp,pressure,z_factor,GFR,rhogas,rhooil,Cd0,dm)



print('OUTPUT')
print("The Compessibility factor of gas is equal to -  " ,round(z_factor,3))
print("The Viscocity of gas is equal to -  " ,round(viscogas,3))
print("The density of gas is equal to - " ,round(rhogas,3))
print("The density of oil is equal to - " ,round(rhooil,3))
print("The density of gas is equal to - " ,round(rhooil,3))
print("The gas capacity factor (dLeff) is equal to - " ,round(dLeff,3))
print("The Discharge coefficient after interation - ",round(Cd0,3))


dlist=[16,20,24,28,32,36,40,44,48]
Dlist=[]
Leffgaslist=[]
Leffliqlist=[]
dLefflist2 = []
Lsslist=[]
Slenrlist=[]

 
#Calculate Liquid Capacity Constraint
tr=[1,2,3,4]
length0 = len(tr)


for x in tr:
 print("For Retention time (Tr)= ",x)

 for d in dlist:
   dLeff2 = x*LFR/0.7
   Leffliq=np.sqrt(np.divide(dLeff2,d))
   Leffliqlist.append(Leffliq)

   Lss=Leffliq*4/3
   Lsslist.append(Lss)

   Slenr=12*Lss/d
   Slenrlist.append(Slenr)

   Leffgas=np.divide(dLeff,d)
   Leffgaslist.append(Leffgas)
   print("For the diameter(in) - ",d," the values of Liquid Leff(ft)- ",round(Leffliqlist[-1],2)," The value of Leff for gas(ft)- ",round(Leffgaslist[-1],2)," The value of Liq Lss(ft)-  ",round(Lsslist[-1],2)," The value of Slenderness Ratio- ",round(Slenrlist[-1],2))
  

print("NOTE - The Dimensions with slenderness ratio in between 3-4 is generally preffered")


input the working pressure in PSI-   1000
input the working Temperatue in Farenhite-   60
input the Specifiv gravity of gas-   0.6
input the mole fraction of H2S in gas-   0
input the mole fraction of CO2 in gas-   0
input the Gas Flow Rate in MMscfd -   10
input the iquid Flow Rate in  Bopd-     2000
input the viscocity of gas in Cp-   0.013
input the drop diameter in microns-  140
input the oil API Gravity-  40
The Discharge coefficient after interation -  0.857
The Compessibility factor of gas is equal to -   0.852
The Viscocity of gas is equal to -   0.013
The density of gas is equal to -  3.657
The density of oil is equal to -  51.485
The density of gas is equal to -  51.485
The gas capacity factor (dLeff) is equal to -  40.274
For Retention time (Tr)=  1
For the diameter -  16  the values of Liquid Leff-  13.36  The value of Leff for gas-  2.52  The value of Liq Lss-   17.82  The value of Slenderness Ratio-  13.36
For the diameter -  20  the values of Liquid Leff-  11.95  The val